In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
train_dir = '/content/drive/MyDrive/health_app/brain/brain_classification/Training'
test_dir = '/content/drive/MyDrive/health_app/brain/brain_classification/Testing'

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# Define image transformations
transform = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [22]:
# Load full training data
full_train_data = datasets.ImageFolder(root=train_dir, transform=transform['train'])

# Split into train and validation sets
train_idx, val_idx = train_test_split(
    list(range(len(full_train_data))),
    test_size=0.2,
    stratify=full_train_data.targets,
    random_state=42
)

train_data = Subset(full_train_data, train_idx)
val_data = Subset(full_train_data, val_idx)

# Load test data
test_data = datasets.ImageFolder(root=test_dir, transform=transform['test'])

# Create data loaders
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False, num_workers=2)

# Class names
class_names = full_train_data.classes
print("Classes:", class_names)

Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [23]:
# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Modify the input layer to accept grayscale (1-channel)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the output layer for 4-class classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler (optional but useful)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [25]:
num_epochs = 10  # You can increase based on performance
best_val_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 20)

    # Training phase
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Adjust learning rate if needed
    scheduler.step(val_loss)

    # Save best model (optional)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print("✅ Best model saved!")


Epoch 1/10
--------------------
Train Loss: 0.6143, Train Acc: 0.7698
Val Loss: 0.3452, Val Acc: 0.8696
✅ Best model saved!

Epoch 2/10
--------------------
Train Loss: 0.3620, Train Acc: 0.8724
Val Loss: 0.4085, Val Acc: 0.8696

Epoch 3/10
--------------------
Train Loss: 0.2574, Train Acc: 0.9083
Val Loss: 0.2921, Val Acc: 0.9073
✅ Best model saved!

Epoch 4/10
--------------------
Train Loss: 0.1998, Train Acc: 0.9304
Val Loss: 0.4802, Val Acc: 0.8618

Epoch 5/10
--------------------
Train Loss: 0.1681, Train Acc: 0.9405
Val Loss: 0.1196, Val Acc: 0.9606
✅ Best model saved!

Epoch 6/10
--------------------
Train Loss: 0.1293, Train Acc: 0.9560
Val Loss: 0.5403, Val Acc: 0.8705

Epoch 7/10
--------------------
Train Loss: 0.1328, Train Acc: 0.9571
Val Loss: 0.1774, Val Acc: 0.9466

Epoch 8/10
--------------------
Train Loss: 0.0714, Train Acc: 0.9748
Val Loss: 0.1073, Val Acc: 0.9659
✅ Best model saved!

Epoch 9/10
--------------------
Train Loss: 0.0832, Train Acc: 0.9691
Val Loss:

In [26]:
# Load best model
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

test_loss, test_correct, test_total = 0.0, 0, 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        test_loss += loss.item() * inputs.size(0)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

final_test_loss = test_loss / test_total
final_test_acc = test_correct / test_total

print(f"\n📈 Test Loss: {final_test_loss:.4f}, Test Accuracy: {final_test_acc:.4f}")


📈 Test Loss: 0.0933, Test Accuracy: 0.9687
